# Necessary Upgrades
Run the cells in this section to make sure you have the latest version of sklearn and joblib.

Restart your kernel after installing.

In [1]:
## Update sklearn to prevent version mismatches
# !pip install sklearn --upgrade

In [2]:
## install joblib. This will be used to save your model. 
# !pip install joblib
## Restart your kernel after installing 

# Import Dependencies

In [3]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

import joblib

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("data/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
# df.head()

In [21]:
# drop the error columns
df = df.loc[:, ~df.columns.str.contains('_err')]
df.columns

# column definitions is available at the end of this notebook.

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,...,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,162.513840,0.586,4.50700,874.8,...,443,9.11,25.8,2,5455,4.467,0.927,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,175.850252,0.969,1.78220,10829.0,...,638,39.30,76.3,1,5853,4.544,0.868,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,170.307565,1.276,2.40641,8079.2,...,1395,891.96,505.6,1,5805,4.564,0.791,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,171.595550,0.701,1.65450,603.3,...,1406,926.16,40.9,1,6031,4.438,1.046,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,172.979370,0.762,3.14020,686.0,...,1160,427.65,40.2,2,6046,4.486,0.972,296.28613,48.224670,15.714


# Select your features (columns) and y-variable.
Column definitions are available at the end of this notebook.

Use `koi_disposition` for the y values

In [6]:
y = df[["koi_disposition"]]

In [7]:
# test with a subset
# selected_features = df[['koi_fpflag_nt', 'koi_prad', 'koi_teq', 'koi_srad']]

# all the columns minus y
selected_features = df.drop(columns=["koi_disposition"])
# selected_features.head()

# Create a Train Test Split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(selected_features, y, random_state=42, stratify = y)
# X_train.head()

# Pre-processing

Scale the data and perform some feature selection

In [9]:
# Scale your data

X_standard_scaler = StandardScaler().fit(X_train)
X_minmax_scaler = MinMaxScaler().fit(X_train)

X_train_minmax_scaled = X_minmax_scaler.transform(X_train)
X_test_minmax_scaled = X_minmax_scaler.transform(X_test)

X_train_standard_scaled = X_standard_scaler.transform(X_train)
X_test_standard_scaled = X_standard_scaler.transform(X_test)

# Train the Model

In [10]:
logistic_minmax = LogisticRegression()
logistic_minmax.fit(X_train_minmax_scaled, y_train)

logistic_standard = LogisticRegression()
logistic_standard.fit(X_train_standard_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
print(f"Minmax Training Data Score: {logistic_minmax.score(X_train_minmax_scaled, y_train)}")
print(f"Minmax Testing Data Score: {logistic_minmax.score(X_test_minmax_scaled, y_test)}")

minmax_predictions = logistic_minmax.predict(X_test_minmax_scaled)
print(classification_report(y_test, minmax_predictions))

print ("---")

print(f"Standard Training Data Score: {logistic_standard.score(X_train_standard_scaled, y_train)}")
print(f"Standard Testing Data Score: {logistic_standard.score(X_test_standard_scaled, y_test)}")

standard_predictions = logistic_standard.predict(X_test_standard_scaled)
print(classification_report(y_test, standard_predictions))

Minmax Training Data Score: 0.807743658210948
Minmax Testing Data Score: 0.8163615560640732
                precision    recall  f1-score   support

     CANDIDATE       0.65      0.56      0.60       422
     CONFIRMED       0.63      0.70      0.67       450
FALSE POSITIVE       0.99      1.00      0.99       876

      accuracy                           0.82      1748
     macro avg       0.76      0.75      0.75      1748
  weighted avg       0.81      0.82      0.81      1748

---
Standard Training Data Score: 0.822429906542056
Standard Testing Data Score: 0.8289473684210527
                precision    recall  f1-score   support

     CANDIDATE       0.68      0.59      0.63       422
     CONFIRMED       0.66      0.72      0.69       450
FALSE POSITIVE       0.99      1.00      0.99       876

      accuracy                           0.83      1748
     macro avg       0.77      0.77      0.77      1748
  weighted avg       0.83      0.83      0.83      1748



# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [12]:
# Create the GridSearchCV model

param_grid = {'C': [1, 5, 10],
              'penalty': ["l1", "l2"]}

logistic_minmax_search = LogisticRegression()
logistic_grid_minmax = GridSearchCV(logistic_minmax_search, param_grid, cv=5, verbose=2)

logistic_standard_search = LogisticRegression()
logistic_grid_standard = GridSearchCV(logistic_standard_search, param_grid, cv=5, verbose=2)

In [13]:
# Train the model with GridSearch
best_logistic_model_minmax = logistic_grid_minmax.fit(X_train_minmax_scaled, y_train)
best_logistic_model_standard = logistic_grid_standard.fit(X_train_standard_scaled, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] C=1, penalty=l1 .................................................
[CV] .................................. C=1, penalty=l1, total=   0.0s
[CV] C=1, penalty=l1 .................................................
[CV] .................................. C=1, penalty=l1, total=   0.0s
[CV] C=1, penalty=l1 .................................................
[CV] .................................. C=1, penalty=l1, total=   0.0s
[CV] C=1, penalty=l1 .................................................
[CV] .................................. C=1, penalty=l1, total=   0.0s
[CV] C=1, penalty=l1 .................................................
[CV] .................................. C=1, penalty=l1, total=   0.0s
[CV] C=1, penalty=l2 .................................................
[CV] .................................. C=1, penalty=l2, total=   0.1s
[CV] C=1, penalty=l2 .................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] .................................. C=1, penalty=l2, total=   0.1s
[CV] C=1, penalty=l2 .................................................
[CV] .................................. C=1, penalty=l2, total=   0.1s
[CV] C=1, penalty=l2 .................................................
[CV] .................................. C=1, penalty=l2, total=   0.1s
[CV] C=1, penalty=l2 .................................................
[CV] .................................. C=1, penalty=l2, total=   0.1s
[CV] C=5, penalty=l1 .................................................
[CV] .................................. C=5, penalty=l1, total=   0.0s
[CV] C=5, penalty=l1 .................................................
[CV] .................................. C=5, penalty=l1, total=   0.0s
[CV] C=5, penalty=l1 .................................................
[CV] .................................. C=5, penalty=l1, total=   0.0s
[CV] C=5, penalty=l1 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    1.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] .................................. C=1, penalty=l2, total=   0.1s
[CV] C=1, penalty=l2 .................................................
[CV] .................................. C=1, penalty=l2, total=   0.1s
[CV] C=1, penalty=l2 .................................................
[CV] .................................. C=1, penalty=l2, total=   0.1s
[CV] C=1, penalty=l2 .................................................
[CV] .................................. C=1, penalty=l2, total=   0.1s
[CV] C=1, penalty=l2 .................................................
[CV] .................................. C=1, penalty=l2, total=   0.1s
[CV] C=5, penalty=l1 .................................................
[CV] .................................. C=5, penalty=l1, total=   0.0s
[CV] C=5, penalty=l1 .................................................
[CV] .................................. C=5, penalty=l1, total=   0.0s
[CV] C=5, penalty=l1 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    1.6s finished


In [14]:
print(f"Minmax best params: {best_logistic_model_minmax.best_params_}")
print(f"Minmax best score: {best_logistic_model_minmax.best_score_}")
print(f"Minmax best best estimator: {best_logistic_model_minmax.best_estimator_}")
print ("---")
print(f"Standard best params: {best_logistic_model_standard.best_params_}")
print(f"Standard best score: {best_logistic_model_standard.best_score_}")
print(f"Standard best best estimator: {best_logistic_model_standard.best_estimator_}")

Minmax best params: {'C': 10, 'penalty': 'l2'}
Minmax best score: 0.8104157722003507
Minmax best best estimator: LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
---
Standard best params: {'C': 10, 'penalty': 'l2'}
Standard best score: 0.8224315778749662
Standard best best estimator: LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


# Test Model

In [15]:
print(f"Minmax Training Data Score: {best_logistic_model_minmax.score(X_train_minmax_scaled, y_train)}")
print(f"Minmax Testing Data Score: {best_logistic_model_minmax.score(X_test_minmax_scaled, y_test)}")

minmax_predictions = best_logistic_model_minmax.predict(X_test_minmax_scaled)
print(classification_report(y_test, minmax_predictions))

print ("---")

print(f"Standard Training Data Score: {best_logistic_model_standard.score(X_train_standard_scaled, y_train)}")
print(f"Standard Testing Data Score: {best_logistic_model_standard.score(X_test_standard_scaled, y_test)}")

standard_predictions = best_logistic_model_standard.predict(X_test_standard_scaled)
print(classification_report(y_test, standard_predictions))

Minmax Training Data Score: 0.8148006866297921
Minmax Testing Data Score: 0.8283752860411899
                precision    recall  f1-score   support

     CANDIDATE       0.68      0.59      0.63       422
     CONFIRMED       0.66      0.72      0.69       450
FALSE POSITIVE       0.99      1.00      0.99       876

      accuracy                           0.83      1748
     macro avg       0.77      0.77      0.77      1748
  weighted avg       0.83      0.83      0.83      1748

---
Standard Training Data Score: 0.8247186725157353
Standard Testing Data Score: 0.834096109839817
                precision    recall  f1-score   support

     CANDIDATE       0.69      0.60      0.65       422
     CONFIRMED       0.66      0.73      0.70       450
FALSE POSITIVE       0.99      1.00      0.99       876

      accuracy                           0.83      1748
     macro avg       0.78      0.78      0.78      1748
  weighted avg       0.83      0.83      0.83      1748



In [16]:
# see how it compares
minmax_predicted = minmax_predictions[:20]
standard_predicted = standard_predictions[:20]
actual = y_test["koi_disposition"][:20].tolist()
pd.DataFrame({"Actual": actual, "minmax scaler": minmax_predicted, "standard scaler": standard_predicted}).reset_index(drop=True)

,Actual,minmax scaler,standard scaler
0,CANDIDATE,CANDIDATE,CANDIDATE
1,FALSE POSITIVE,FALSE POSITIVE,FALSE POSITIVE
2,FALSE POSITIVE,FALSE POSITIVE,FALSE POSITIVE
3,FALSE POSITIVE,FALSE POSITIVE,FALSE POSITIVE
4,CANDIDATE,CONFIRMED,CANDIDATE
5,FALSE POSITIVE,FALSE POSITIVE,FALSE POSITIVE
6,CANDIDATE,CANDIDATE,CANDIDATE
7,FALSE POSITIVE,FALSE POSITIVE,FALSE POSITIVE
8,FALSE POSITIVE,FALSE POSITIVE,FALSE POSITIVE
9,FALSE POSITIVE,FALSE POSITIVE,FALSE POSITIVE


# Save the Model

In [19]:
minmax_logistic = 'models/logistic_minmax.h5'
joblib.dump(best_logistic_model_minmax, minmax_logistic)

standard_logistic = 'models/logistic_standard.h5'
joblib.dump(best_logistic_model_standard, standard_logistic)

['models/logistic_standard.h5']

In [20]:
loaded_model = joblib.load("models/logistic_minmax.h5")
print(f"{loaded_model.score(X_test_minmax_scaled, y_test)}")

loaded_model = joblib.load("models/logistic_standard.h5")
print(f"{loaded_model.score(X_test_standard_scaled, y_test)}")

0.8283752860411899
0.834096109839817


# Column Details

### Exoplanet Archive Information
* 'koi_disposition': The pipeline flag that designates the most probable physical explanation of the KOI

### Project Disposition Columns
* 'koi_fpflag_nt': A KOI whose light curve is not consistent with that of a transiting planet.
* 'koi_fpflag_ss': A KOI that is observed to have a significant secondary event, transit shape, or out-of-eclipse variability, which indicates that the transit-like event is most likely caused by an eclipsing binary.
* 'koi_fpflag_co': The source of the signal is from a nearby star, as inferred by measuring the centroid location of the image both in and out of transit, or by the strength of the transit signal in the target's outer (halo) pixels as compared to the transit signal from the pixels in the optimal (or core) aperture.
* 'koi_fpflag_ec': The KOI shares the same period and epoch as another object and is judged to be the result of flux contamination in the aperture or electronic crosstalk.

### Transit Properties
Transit parameters delivered by the Kepler Project are typically best-fit parameters produced by a Mandel-Agol (2002) fit to a multi-quarter Kepler light curve, assuming a linear orbital ephemeris. Some of the parameters listed below are fit directly, other are derived from the best-fit parameters. Limb-darkening coefficients are fixed and pre-calculated from host star properties. Orbital Period, Transit Epoch, Planet-Star Radius Ratio, Planet-Star Distance over Star Radius and Impact Parameter are the free parameters in the fit. Matrix covariances are adopted as errors to the fit parameters; they therefore ignore the effects of correlation between the fit parameters and are likely to be underestimates.

See the links in the Purpose of KOI document for each activity table for more details on the fits for each delivery.

Scaled planetary parameters combine the dimensionless fit parameters with physical stellar parameters to produce planet characteristics in physical units.

Best-fit planetary transit parameters are typically normalized to the size of the host star. Physical planet parameters may be derived by scaling to the star's size and temperature. Transit parameters also depend weakly upon the limb-darkening coefficients which are derived from the stellar parameters (e.g., Claret and Bloemen 2011).

* 'koi_period': The interval between consecutive planetary transits.
* 'koi_time0bk': The time corresponding to the center of the first detected transit in Barycentric Julian Day (BJD) minus a constant offset of 2,454,833.0 days. The offset corresponds to 12:00 on Jan 1, 2009 UTC.
* 'koi_impact': The sky-projected distance between the center of the stellar disc and the center of the planet disc at conjunction, normalized by the stellar radius
* 'koi_duration': The duration of the observed transits. Duration is measured from first contact between the planet and star until last contact. Contact times are typically computed from a best-fit model produced by a Mandel-Agol (2002) model fit to a multi-quarter Kepler light curve, assuming a linear orbital ephemeris.
* 'koi_depth': The fraction of stellar flux lost at the minimum of the planetary transit. Transit depths are typically computed from a best-fit model produced by a Mandel-Agol (2002) model fit to a multi-quarter Kepler light curve, assuming a linear orbital ephemeris.
* 'koi_prad': The radius of the planet. Planetary radius is the product of the planet star radius ratio and the stellar radius.
* 'koi_teq': Approximation for the temperature of the planet. 
* 'koi_insol': Insolation flux is another way to give the equilibrium temperature.

### Threshold-Crossing Event (TCE) Information
The Transiting Planet Search (TPS) module of the Kepler data analysis pipeline performs a detection test for planet transits in the multi-quarter, gap-filled flux time series. The TPS module detrends each quarterly PDC light curve to remove edge effects around data gaps and then combines the data segments together, filling gaps with interpolated data so as to condition the flux time series for a matched filter. The module applies an adaptive, wavelet-based matched filter (Jenkins 2002, Jenkins et al. 2010 and Tenenbaum et al. (2012)) to perform a joint characterization of observation noise and detection of transit-like features in the light curve.

The TPS module estimates the Power Spectral Density of the flux time series as a function in time. This provides coefficients for a whitening filter to accommodate non-stationary, non-white noise and yields Single Event Statistic (SES) time series components. These can be interpreted as measurements of the statistical significance of the presence of a transit of trial duration at each point in the time series.

Single Event Statistics are folded at each trial orbital period and the maximum Multiple Event Statistic (MES) is obtained over all trial periods and phases. The MES estimates the signal to noise ratio of the putative transit-like sequence against the measurement noise. The MES threshold for defining the sample of Threshold Crossing Events (TCEs) is provided within the Release Notes. For reference, a lower MES threshold of 7.1σ limits the number of false positives in the TCE sample due to statistical random noise to less than 1 over the primary mission (Jenkins, Caldwell and Borucki 2002).

* 'koi_model_snr': Transit depth normalized by the mean uncertainty in the flux during the transits.
* 'koi_tce_plnt_num': TCE Planet Number federated to the KOI.

### Stellar Parameters
Stellar effective temperature, surface gravity, metallicity, radius, mass, and age should comprise a consistent set. Associated error estimates are 1-σ uncertainties.

* 'koi_steff': The photospheric temperature of the star.
* 'koi_slogg': The base-10 logarithm of the acceleration due to gravity at the surface of the star.
* 'koi_srad': The photospheric radius of the star

### KIC Parameters
* 'ra': KIC Right Ascension
* 'dec': KIC Declination
* 'koi_kepmag': Kepler-band (mag)